## Expert Evaluation
Conduct the expert evaluation by importing the json files from `data/3_submissions_with_categorized_feedback_jsons` into a new expert evaluation in the playground. Download the results of the completed expert evaluation as well as the evaluation configuration.
Save the downloaded files in the `data/4_expert_evaluation/input_pseudonymized` directory.

### (Optional) Splitting the Evaluation Config
If you want to do multiple, potentially incomplete evaluations, you can split the evaluation config into multiple configs. All configs will contain the same exercises, submissions, feedback suggestions, and metrics. However, the configs will have different exercise and submission orders.
Specifically, the configs will have a rotating order of exercises and either an ascending or descending order of submissions.

This allows different experts to evaluate the same exercises and submissions, but in a different order. This can help reduce bias in the evaluation process. Additionally, it enables the experts to hand in a incomplete evaluation early, therefore reducing the invested time per expert.

In [ ]:
import json
import os
import copy

evaluation_config_path = "data/4_expert_evaluation/evaluation_config.json"
evaluation_config_splitted_path = "data/4_expert_evaluation/evaluation_configs_splitted"

os.makedirs(evaluation_config_splitted_path, exist_ok=True)

evaluation_config = {}

with open(evaluation_config_path, "r") as file:
    evaluation_config = json.load(file)

evaluation_config["started"] = False
evaluation_config["expertIds"] = []
evaluation_config["mappings"] = {}

exercises = evaluation_config["exercises"]


def generate_config(starting_index: int, ascending: bool) -> None:
    new_config = copy.deepcopy(evaluation_config)
    new_config["name"] = (
        f"{evaluation_config['name']}_{'ascending' if ascending else 'descending'}_{starting_index}"
    )
    new_config["exercises"] = exercises[starting_index:] + exercises[:starting_index]

    for exercise in new_config["exercises"]:
        exercise["submissions"].sort(key=lambda x: x["id"], reverse=not ascending)

    new_config_path = f"{evaluation_config_splitted_path}/{new_config['name']}.json"
    with open(new_config_path, "w") as new_file:
        json.dump(new_config, new_file, indent=4)


for ascending in [True, False]:
    for starting_index in range(len(exercises)):
        generate_config(starting_index, ascending)

### Find Evaluation Files
Once you have saved the expert evaluation results and the config files in the `data/4_expert_evaluation/input_pseudonymized` directory, run the following cell to create lists of all config and progress files.

In [ ]:
from service.json_service import get_evaluation_files

data_dir = "data/4_expert_evaluation/input_pseudonymized"
output_dir = "data/4_expert_evaluation/output_depseudonymized"

evaluation_config_files, evaluation_progress_files = get_evaluation_files(data_dir)

print(f"Found {len(evaluation_config_files)} evaluation config files.")
print(f"Found {len(evaluation_progress_files)} evaluation progress files.")

### Create a Common Config File
Given multiple config files in the `data/4_expert_evaluation/input_pseudonymized` directory, creates a common config applicable to all progress files and saves it to the `data/4_expert_evaluation/output_depseudonymized` directory.

In [ ]:
import json
import os
from service.json_service import create_common_config

common_evaluation_config = create_common_config(evaluation_config_files)

common_config_path = f"{output_dir}/common_evaluation_config.json"
os.makedirs(output_dir, exist_ok=True)
with open(common_config_path, "w") as common_file:
    json.dump(common_evaluation_config, common_file, indent=4)

print(f"Created Common Evaluation Config With:")
print(f"- {len(common_evaluation_config.get("exercises", []))} Exercises")
print(f"- {sum([len(e.get("submissions", [])) for e in common_evaluation_config.get("exercises", [])])} Submissions")
print(f"- {len(common_evaluation_config.get("metrics", []))} Metrics")

### Resolve Feedback Types and Metric Titles in Progress Files
Resolves the feedback types and Metric Titles for the progress files in the `data/4_expert_evaluation/input_pseudonymized` directory and saves the de-pseudonymized progress files in the `data/4_expert_evaluation/output_depseudonymized` directory.

In [ ]:
import json
import os
from service.json_service import resolve_feedback_types_and_metric_titles

metric_ids_to_titles = {metric["id"]: metric["title"] for metric in common_evaluation_config.get("metrics", [])}
pseudonym_to_feedback_type = common_evaluation_config.get("mappings", {})

for progress_file, progress_data in resolve_feedback_types_and_metric_titles(evaluation_progress_files, metric_ids_to_titles, pseudonym_to_feedback_type).items():
    output_progress_path = f"{output_dir}/{os.path.basename(progress_file)}"
    with open(output_progress_path, "w") as output_file:
        json.dump(progress_data, output_file, indent=4)